---
title: Effortless Bayesian Deep Learning through Laplace Redux
subtitle: JuliaCon 2022
author: Patrick Altmeyer
format: 
  revealjs:
    logo: /www/images/delft_logo.png
    footer: |
      Effortless Bayesian Deep Learning through Laplace Redux -- JuliaCon 2022 -- Patrick Altmeyer
    self-contained: true
    smaller: true
    scrollable: true
    preview-links: auto
    slide-number: true
    transition: slide
    background-transition: fade
    fig-align: center
bibliography: https://raw.githubusercontent.com/pat-alt/bib/main/bib.bib
execute:
  eval: false
  echo: true
---

```{julia}
#| echo: false
using Pkg; Pkg.activate("dev")
using Plots, PlotThemes
theme(:wong)
include("dev/utils.jl")
www_path = "dev/resources/juliacon22/www"
```


## Overview

:::{.incremental}
- The Case for Bayesian Deep Learning
- Laplace Redux in Julia 📦
    - From Bayesian Logistic Regression ...
    - ... to Bayesian Neural Networks.
- Goals and Ambitions 🎯
:::

# The Case for Bayesian Deep Learning

## Bayesian Model Averaging

> Don't put all your 🥚 in one 🧺.


:::{.incremental}
- In Deep Learning we typically maximise highly non-convex functions full of local optima and saddle points.
- There may be many $\hat\theta_1, ..., \hat\theta_m$ that are slightly different, but yield similar performance.
:::

. . .

> [...] parameters correspond to a diverse variety of compelling explanations for the data. 
> [@wilson2020case]

. . .

$\theta$ is a random variable. Shouldn't we treat it that way?

$$
p(y|x,\mathcal{D}) = \int p(y|x,\theta)p(\theta|\mathcal{D})d\theta
$$ {#eq-bma}

> Intractable!

. . .

In practice we typically rely on a **plugin** approximation [@murphy2022probabilistic].

$$
p(y|x,\mathcal{D}) = \int p(y|x,\theta)p(\theta|\mathcal{D})d\theta \approx p(y|x,\hat\theta)
$$ {#eq-plugin}

> Yes, "plugin" is literal ... can we do better?

## Enter: Bayesian Deep Learning 🔮

> Yes, we can! 

::::{.columns}

:::{.column width="50%"}

#### Popular approaches include ...

:::{.fragment .semi-fade-out fragment-index=4}

:::{.fragment .fade-left fragment-index=0}
MCMC (see [`Turing`](https://turing.ml/dev/tutorials/03-bayesian-neural-network/))
:::

:::{.fragment .fade-left fragment-index=1}
Variational Inference [@blundell2015weight]
:::

:::{.fragment .fade-left fragment-index=2}
Monte Carlo Dropout [@gal2016dropout]
:::

:::{.fragment .fade-left fragment-index=3}
Deep Ensembles [@lakshminarayanan2016simple]
:::

:::

:::{.fragment .fade-left fragment-index=5}
Laplace Redux (@immer2020improving,@daxberger2021laplace)
:::

. . .

![Pierre-Simon Laplace as chancellor of the Senate under the First French Empire. Source: [Wikipedia](https://en.wikipedia.org/wiki/Pierre-Simon_Laplace)](/www/images/laplace_dude.jpeg){#fig-laplace-dude width="25%"}

:::

:::{.column width="50%"}
![Simulation of changing posteriour predictive distribution. Image by author.](/www/images/intro.gif){#fig-intro}
:::

::::

## Laplace Approximation

> We first need to estimate the weight posterior $p(\theta|\mathcal{D})$ ...

. . .

**Idea** 💡: Taylor approximation at the mode.

:::{.incremental}
- Going through the maths we find that this yields a Gaussian posteriour centered around the MAP estimate $\hat\theta$ (see pp. 148/149 in @murphy2022probabilistic).
- Covariance corresponds to inverse Hessian at the mode (in practice we may have to rely on approximations).
:::

. . .

![Unnormalized log-posterior and corresponding Laplace Approximation. Source: @murphy2022probabilistic.](/www/images/laplace_posterior.png)

> Now we can rely on MC or Probit Approximation to compute posterior predictive (classification).

# Laplace Redux in Julia 

## [`LaplaceRedux.jl`](https://github.com/juliatrustworthyai/LaplaceRedux.jl) - a small package 📦

[![Dev](https://img.shields.io/badge/docs-dev-blue.svg)](https://juliatrustworthyai.github.io/LaplaceRedux.jl/dev) [![Build Status](https://github.com/juliatrustworthyai/LaplaceRedux.jl/actions/workflows/CI.yml/badge.svg?branch=main)](https://github.com/juliatrustworthyai/LaplaceRedux.jl/actions/workflows/CI.yml?query=branch%3Amain) [![Coverage](https://codecov.io/gh/juliatrustworthyai/LaplaceRedux.jl/branch/main/graph/badge.svg)](https://codecov.io/gh/juliatrustworthyai/LaplaceRedux.jl)

> What started out as my first coding project Julia ... 

:::{.incremental}
- Big fan of **learning by coding** so after reading the first chapters of @murphy2022probabilistic I decided to code up Bayesian Logisitic Regression from scratch. 
- I also wanted to learn Julia at the time, so tried to hit two birds with one stone. 
- **Outcome**: 1. This [blog post](https://towardsdatascience.com/bayesian-logistic-regression-53df017ba90f). 2. I have since been hooked on Julia. 
:::

. . .

> ... has turned into a small package 📦  with great potential.

:::{.incremental}
- When coming across the NeurIPS 2021 paper on Laplace Redux for deep learning [@daxberger2021laplace], I figured I could step it up a notch.
- **Outcome**: `LaplaceRedux.jl` and another [blog post](https://towardsdatascience.com/go-deep-but-also-go-bayesian-ab25efa6f7b).
:::

. . . 

So let's add the package ...

```julia
using Pkg
Pkg.add("https://github.com/juliatrustworthyai/LaplaceRedux.jl")
```

... and use it. 


```{julia}
using LaplaceRedux
```


## From Bayesian Logistic Regression ...

::::{.columns}::::
:::{.column width='50%'}
#### From maths ...

. . .

We assume a Gaussian prior for our weights ...
$$
p(\theta) \sim \mathcal{N} \left( \theta | \mathbf{0}, \lambda^{-1} \mathbf{I} \right)=\mathcal{N} \left( \theta | \mathbf{0}, \mathbf{H}_0^{-1} \right)
$$ {#eq-prior}

. . .

... which corresponds to logit binary crossentropy loss with weight decay:

$$
\ell(\theta)= - \sum_{n}^N [y_n \log \mu_n + (1-y_n)\log (1-\mu_n)] + \\ \frac{1}{2} (\theta-\theta_0)^T\mathbf{H}_0(\theta-\theta_0)
$$ {#eq-loss}

. . .

For Logistic Regression we have the Hessian in closed form (p. 338 in @murphy2022probabilistic):

$$
\nabla_{\theta}\nabla_{\theta}^\mathsf{T}\ell(\theta) = \frac{1}{N} \sum_{n}^N(\mu_n(1-\mu_n)\mathbf{x}_n)\mathbf{x}_n^\mathsf{T} + \mathbf{H}_0
$$ {#eq-hessian}

:::
:::{.column width='50%'}
#### ... to code
. . . 

```julia
# Hessian:
function ∇∇𝓁(θ,θ_0,H_0,X,y)
    N = length(y)
    μ = sigmoid(θ,X)
    H = ∑(μ[n] * (1-μ[n]) * X[n,:] * X[n,:]' for n=1:N)
    return H + H_0
end
```

> Gotta love Julia ❤️💜💚

. . .


```{julia}
#| echo: false
# Import libraries.
using Plots, PlotThemes, Statistics, LaplaceRedux
theme(:wong)
# Number of points to generate.
xs, y = LaplaceRedux.Data.toy_data_linear(100)
X = hcat(xs...); # bring into tabular format
data = zip(xs,y);
```


Logistic Regression can be done in `Flux` ...


```{julia}
using Flux
# Initializing weights as zeros only for illustrative purposes:
nn = Chain(Dense(zeros(1,2),zeros(1))) 
```


. . .


```{julia}
#| echo: false
λ = 0.5
sqnorm(x) = sum(abs2, x)
weight_regularization(λ=λ) = 1/2 * λ^2 * sum(sqnorm, Flux.params(nn))
loss(x, y; λ=λ) = Flux.Losses.logitbinarycrossentropy(nn(x), y) + weight_regularization(λ)
```

```{julia}
#| echo: false
using Flux.Optimise: update!, ADAM
opt = ADAM()
epochs = 50

for epoch = 1:epochs
  for d in data
    gs = gradient(Flux.params(nn)) do
      l = loss(d...)
    end
    update!(opt, Flux.params(nn), gs)
  end
end
```


... but now we autograd! Leveraged in `LaplaceRedux`.


```{julia}
la = Laplace(nn, λ=λ)
fit!(la, data)
```

```{julia}
#| echo: false
p_plugin = plot_contour(X',y,la;title="Plugin",type=:plugin);
p_laplace = plot_contour(X',y,la;title="Laplace")
# Plot the posterior distribution with a contour plot.
plt = plot(p_plugin, p_laplace, layout=(1,2), size=(1000,400))
savefig(plt, joinpath(www_path,"posterior_predictive_logit.png"))
```


![Posterior predictive distribution of Logistic regression in the 2D feature space using plugin estimator (left) and Laplace approximation (right). Image by author.](/www/images/posterior_predictive_logit.png){#fig-pred-logit width="70%"}

:::
::::


```{julia}
#| echo: false
Λ = [1e10, 0.5, 1e-10]
plts = []
nns = []
for λ ∈ Λ
  nn = Chain(Dense(zeros(1,2),zeros(1)))
  # train
  epochs = 50
  for epoch = 1:epochs
    for d in data
      gs = gradient(Flux.params(nn)) do
        l = loss(d...;λ=λ)
      end
      update!(opt, Flux.params(nn), gs)
    end
  end
  # laplace
  la = Laplace(nn, λ=λ)
  fit!(la, data)  
  # plot 
  plt = plot_contour(X',y,la;title="σ₀²=$(1/λ)")
  plts = vcat(plts..., plt)
  nns = vcat(nns..., nn)
end
plt = plot(plts..., layout=(1,3), size=(1200,300))
savefig(plt, joinpath(www_path,"posterior_predictive_prior_logit.png"))
```


## ... to Bayesian Neural Networks

::::{.columns}::::
:::{.column width='30%'}

```{julia}
#| echo: false
# Number of points to generate:
xs, y = LaplaceRedux.Data.toy_data_non_linear(200)
X = hcat(xs...); # bring into tabular format
data = zip(xs,y)
```


#### Code

. . .

An actual MLP ...


```{julia}
# Build MLP:
n_hidden = 32
D = size(X)[1]
nn = Chain(
    Dense(
      randn(n_hidden,D)./10,
      zeros(n_hidden), σ
    ),
    Dense(
      randn(1,n_hidden)./10,
      zeros(1)
    )
)  
```

```{julia}
#| echo: false
λ = 0.01
sqnorm(x) = sum(abs2, x)
weight_regularization(λ=λ) = 1/2 * λ^2 * sum(sqnorm, Flux.params(nn))
loss(x, y; λ=λ) = Flux.Losses.logitbinarycrossentropy(nn(x), y) + weight_regularization(λ)
```

```{julia}
#| echo: false
# Training:
epochs = 200
for epoch = 1:epochs
  for d in data
    gs = gradient(Flux.params(nn)) do
      l = loss(d...)
    end
    update!(opt, Flux.params(nn), gs)
  end
end
```


. . .

... same API call:


```{julia}
la = Laplace(
  nn, λ=λ, 
  subset_of_weights=:last_layer
)
fit!(la, data)
```

```{julia}
#| echo: false
p_plugin = plot_contour(X',y,la;title="Plugin",type=:plugin)
p_laplace = plot_contour(X',y,la;title="Laplace")
# Plot the posterior distribution with a contour plot.
plt = plot(p_plugin, p_laplace, layout=(1,2), size=(1000,400))
savefig(plt, joinpath(www_path,"posterior_predictive_mlp.png"))
```


. . .

:::
:::{.column width='70%'}
#### Results

. . .

![Posterior predictive distribution of MLP in the 2D feature space using plugin estimator (left) and Laplace approximation (right). Image by author.](/www/images/posterior_predictive_mlp.png){#fig-pred-mlp width="100%"}
:::
::::

## A quick note on the prior 


```{julia}
#| echo: false
Λ = [1e10, 0.01, 1e-10]
plts = []
nns = []
for λ ∈ Λ
  nn = Chain(
      Dense(randn(n_hidden,D)./10,zeros(n_hidden), σ),
      Dense(randn(1,n_hidden)./10,zeros(1))
  )  
  # train
  epochs = 200
  for epoch = 1:epochs
    for d in data
      gs = gradient(Flux.params(nn)) do
        l = loss(d...;λ=λ)
      end
      update!(opt, Flux.params(nn), gs)
    end
  end
  # laplace
  la = Laplace(nn, λ=λ)
  fit!(la, data)  
  # plot 
  plt = plot_contour(X',y,la;title="σ₀²=$(1/λ)")
  plts = vcat(plts..., plt)
  nns = vcat(nns..., nn)
end
plt = plot(plts..., layout=(1,3), size=(1200,300))
savefig(plt, joinpath(www_path,"posterior_predictive_prior.png"))
```



Low prior uncertainty $\rightarrow$ posterior dominated by prior. High prior uncertainty $\rightarrow$ posterior approaches MLE.

#### Logistic Regression

![Prior uncertainty increases from left to right (Logsitic Regression). Image by author.](/www/images/posterior_predictive_prior_logit.png){#fig-pred-prior-logit width="75%"}

#### MLP

![Prior uncertainty increases from left to right (MLP). Image by author.](/www/images/posterior_predictive_prior.png){#fig-pred-prior width="75%"}


## A crucial detail I skipped

> We're really been using linearized neural networks ...

. . .

::::{.columns}

:::{.column width="50%"}
#### MC fails

:::{.incremental}
- Could do Monte Carlo for true BNN predictive, but this performs poorly when using approximations for the Hessian.
- Instead we rely on **linear expansion** of predictive around mode [@immer2020improving].
- **Intuition**: Hessian approximation involves linearization, then so should the predictive. 
:::

. . .

> Applying the GNN approximation [...] turns the underlying probabilistic model locally from a BNN into a GLM [...] Because we have effectively done inference in the GGN-linearized model, we should instead predict using these modified features.
> --- @immer2020improving

:::

:::{.column width="50%"}
![MC samples from the Laplace posterior [@lawrence2001variational].](/www/images/mc_fail.png){#fig-mc-fail}
:::

::::

# Goals and Ambitions 🎯

## JuliaCon 2022 and beyond

::::{.columns}

:::{.column width="50%"}

:::{.fragment .semi-fade-out fragment-index=4}
#### To JuliaCon ...

:::{.fragment .strike fragment-index=1}
Learn about Laplace Redux by implementing it in Julia.
:::

:::{.fragment .strike fragment-index=2}
Turn code into a small package. 
:::

:::{.fragment .strike fragment-index=3}
Submit to [JuliaCon 2022](https://juliacon.org/2022/) and share the idea.
:::
:::

#### ... and beyond

. . .

Package is bare-bones at this point and needs a lot of work.

:::{.incremental}
- **Goal**: reach same level of maturity as Python [counterpart](https://aleximmer.github.io/Laplace/). (Beautiful work btw!)
- **Problem**: limited capacity and fairly new to Julia.
- **Solution**: find contributors 🤗.
:::
:::

:::{.column width="50%"}
![Photo by [Ivan Diaz](https://unsplash.com/@ivvndiaz?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText) on [Unsplash](https://unsplash.com/?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText)](/www/images/launch.jpeg){width="70%"}
:::

::::

## Specific Goals

::::{.columns}

:::{.column width="50%"}
#### Easy

- Still missing support for multi-class and regression. 
- Due diligence: peer review and unit testing.

#### Harder

- Hessian approximations still quadratically large: use factorizations.
- Hyperparameter tuning: what about that prior?
- Scaling things up: subnetwork inference.
- Early stopping: do we really end up at the mode?
- ...

:::

:::{.column width="50%"}

![Source: [Giphy](https://giphy.com/gifs/theoffice-nbc-the-office-tv-dUHdTk3tvry9NETa67)](/www/images/dwight.gif)

:::

::::


## More Resources 📚

::::{.columns}

:::{.column width="60%"}
> Read on ...

- Blog post (1) -- Bayesian Logisitic Regression: [[TDS](https://towardsdatascience.com/bayesian-logistic-regression-53df017ba90f), [homepage](https://www.patalt.org/blog/posts/bayesian-logit/)].
- Blog post (2) -- Bayesian Deep Learning: [[TDS](https://towardsdatascience.com/go-deep-but-also-go-bayesian-ab25efa6f7b), [homepage](https://www.patalt.org/blog/posts/effortsless-bayesian-dl/)].
- Detailed slide pack generously shared by [Professor José Miguel Hernández-Lobato](https://jmhl.org/): [[pdf](https://github.com/juliatrustworthyai/LaplaceRedux.jl/blob/main/dev/resources/juliacon22/slides_lobato.pdf)]
- Package [docs](https://www.patalt.org/CounterfactualExplanations.jl/dev/).

> ... or even better: get involved! 🤗

:::

:::{.column width="40%"}

<img src="/www/images/profile.png" height="auto" width="250" style="border-radius:50%; display: block; margin-left: auto; margin-right: auto;">

<div style="text-align: center;">
  <p style="display: inline; vertical-align: middle"> 
    <a href="https://www.linkedin.com/in/patrick-altmeyer-a2a25494/" style="display: inline-block; color: rgb(207, 142, 255) !important;">
      <font style="">
        <img width="60" height="60" src="https://s1g.s3.amazonaws.com/d0fc399dee4218d1e0e0399b8947acab.png" alt="LinkedIn (Personal)" style="border: none; max-width: 100%; height: 60px !important;">
      </font>
    </a>
    <a href="https://twitter.com/paltmey" style="display: inline-block; color: rgb(207, 142, 255) !important;">
      <font style="">
        <img width="60" height="60" src="https://s1g.s3.amazonaws.com/3949237f892004c237021ac9e3182b1d.png" alt="Twitter" style="border: none; max-width: 100%; height: 60px !important;">
      </font>
    </a>
    <a href="https://github.com/pat-alt" style="display: inline-block; color: rgb(207, 142, 255) !important;">
      <font style="">
        <img width="60" height="60" src="https://s1g.s3.amazonaws.com/47f4eb2d0082a8a3611d614b75a09db8.png" alt="Github" style="border: none; max-width: 100%; height: 60px !important;">
      </font>
    </a>
    <a href="https://medium.com/@patrick.altmeyer" style="display: inline-block; color: rgb(207, 142, 255) !important;">
      <font style="">
        <img width="60" height="60" src="https://s1g.s3.amazonaws.com/175f49662614345cb7dbb95fce3f88af.png" alt="Medium" style="border: none; max-width: 100%; height: 60px !important;">
      </font>
    </a>
  </p>
</div>

<img src="/www/images/qr.png" height="auto" width="100" style="display: block; margin-left: auto; margin-right: auto;">
:::

::::

## References 